In [8]:
import requests

# OpenWeatherMap API Key
API_KEY = '9eaf56d61d393059cc8469568754fb35'
BASE_URL = 'https://api.openweathermap.org/data/2.5/weather'

# List of cities with lat and long
cities = [
    {'name': 'City1', 'lat': 40.7128, 'lon': -74.0060},  # example: New York
    {'name': 'City2', 'lat': 34.0522, 'lon': -118.2437}, # example: Los Angeles
    # Add your 100 cities here...
]

def get_average_temp(lat, lon):
    # Build API query
    url = f'{BASE_URL}?lat={lat}&lon={lon}&appid={API_KEY}&units=imperial'  # or 'imperial' for Fahrenheit
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200:
        # Extract temperature data
        temp = data['main']['temp']
        return temp
    else:
        print(f"Error fetching data for lat: {lat}, lon: {lon}")
        return None

# Store temperatures
temps = {}

# Loop through cities and get temps
for city in cities:
    temp = get_average_temp(city['lat'], city['lon'])
    if temp is not None:
        temps[city['name']] = temp

# Print or store results
for city, temp in temps.items():
    print(f"City: {city}, Average Temp: {temp}°C")

City: City1, Average Temp: 72.19°C
City: City2, Average Temp: 72.25°C


In [9]:
import requests
from datetime import datetime, timedelta

# OpenWeatherMap API Key

BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall/timemachine'

# Function to get historical weather for a specific day
def get_daily_weather(lat, lon, date):
    unix_timestamp = int(datetime.timestamp(date))
    url = f"{BASE_URL}?lat={lat}&lon={lon}&dt={unix_timestamp}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to get average temperature for a month
def get_monthly_avg_temp(lat, lon, year, month):
    start_date = datetime(year, month, 1)
    end_date = start_date + timedelta(days=30) if month == 12 else datetime(year, month + 1, 1)
    
    daily_temps = []
    
    # Loop through each day of the month
    current_date = start_date
    while current_date < end_date:
        data = get_daily_weather(lat, lon, current_date)
        if data:
            temp = data['current']['temp']  # 'current' holds the temp data for the specific day
            daily_temps.append(temp)
        current_date += timedelta(days=1)
    
    # Calculate and return monthly average
    if daily_temps:
        avg_temp = sum(daily_temps) / len(daily_temps)
        return avg_temp
    else:
        return None

# Example usage for New York (lat: 40.7128, lon: -74.0060) in January 2023
lat = 40.7128
lon = -74.0060
year = 2023
month = 1

average_temp = get_monthly_avg_temp(lat, lon, year, month)
if average_temp is not None:
    print(f"Average temperature for {year}-{month} in New York: {average_temp:.2f}°C")
else:
    print("Could not retrieve temperature data.")

Error: 401, {"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info."}
Error: 401, {"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info."}
Error: 401, {"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see http

In [14]:
import wikipediaapi
import pandas as pd

# Specify a valid user agent
USER_AGENT = 'Julia Kirk (julia.kirk@colorado.edu)'

# Initialize the Wikipedia API with the custom user agent
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=USER_AGENT
)

# Load your CSV file
cities_df = pd.read_csv('../datasets/data.csv')

# Function to get Wikipedia summary for a city
def get_wiki_summary(city_name):
    city_page = wiki_wiki.page(city_name)
    if city_page.exists():
        return city_page.summary
    else:
        return None

# Apply the function to each city in the CSV
cities_df['Wiki_Summary'] = cities_df['City_State'].apply(get_wiki_summary)

# Save the results to a new CSV file
cities_df.to_csv('cities_with_wiki_data.csv', index=False)

print("Data saved to cities_with_wiki_data.csv")

Data saved to cities_with_wiki_data.csv


In [ ]:
import wikipediaapi
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Specify a valid user agent
USER_AGENT = 'Julia Kirk (julia.kirk@colorado.edu)'

# Initialize the Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=USER_AGENT
)

# Specify the city Wikipedia page
page_name = 'Denver'  # Change this to the city you are analyzing
page = wiki_wiki.page(page_name)

# Fetch the full Wikipedia page
wiki_html = requests.get(page.fullurl, headers={'User-Agent': USER_AGENT}).text

# Parse the page using BeautifulSoup
soup = BeautifulSoup(wiki_html, 'html.parser')

# Find the climate chart, usually it's in a table format
climate_table = soup.find('table', {'class': 'wikitable collapsible'})

# Extract the data from the climate table
climate_data = []
for row in climate_table.find_all('tr'):
    columns = row.find_all('td')
    if len(columns) > 0:
        climate_data.append([col.get_text(strip=True) for col in columns])

# Convert to a DataFrame for easier manipulation
climate_df = pd.DataFrame(climate_data)
print(climate_df)

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
import wikipediaapi
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Specify a valid user agent
USER_AGENT = 'Julia Kirk (julia.kirk@colorado.edu)'

# Initialize the Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=USER_AGENT
)

# Specify the city Wikipedia page
page_name = 'Denver'  # Change this to the city you are analyzing
page = wiki_wiki.page(page_name)

# Check if the page exists
if not page.exists():
    print(f"The page '{page_name}' does not exist.")
else:
    # Fetch the full Wikipedia page
    wiki_html = requests.get(page.fullurl, headers={'User-Agent': USER_AGENT}).text

    # Parse the page using BeautifulSoup
    soup = BeautifulSoup(wiki_html, 'html.parser')

    # Find the climate chart, usually it's in a table format
    climate_table = soup.find('table', {'class': 'wikitable collapsible'})

    if climate_table:
        # Extract the data from the climate table
        climate_data = []
        headers = []

        # Extract headers (if present)
        header_row = climate_table.find('tr')
        for header in header_row.find_all('th'):
            headers.append(header.get_text(strip=True))

        # Extract table data
        for row in climate_table.find_all('tr')[1:]:  # Skip the header
            columns = row.find_all('td')
            if len(columns) > 0:
                climate_data.append([col.get_text(strip=True) for col in columns])

        # Convert to a DataFrame for easier manipulation
        climate_df = pd.DataFrame(climate_data, columns=headers if headers else None)
        print(climate_df)
    else:
        print(f"No climate table found for {page_name}.")

No climate table found for Denver.


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Send a request to the webpage
url = 'https://en.wikipedia.org/wiki/South_Bend,_Indiana'
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to load page {url}")

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Find the table
table = soup.find('table')  # Adjust this to match the specific HTML structure

# Step 4: Extract headers (if available)
headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())

# Step 5: Extract rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the first row if it contains headers
    cells = []
    for td in tr.find_all('td'):
        cells.append(td.text.strip())
    rows.append(cells)

# Step 6: Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Step 7: Save the DataFrame to a CSV file (optional)
df.to_csv('climate_data.csv', index=False)

# Print the DataFrame
print(df)

ValueError: 33 columns passed, passed data had 1 columns

In [20]:
import wikipediaapi
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Specify a valid user agent
USER_AGENT = 'Julia Kirk (julia.kirk@colorado.edu)'

# Initialize the Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=USER_AGENT
)

# Specify the city Wikipedia page
page_name = 'Denver'  # Change this to the city you are analyzing
page = wiki_wiki.page(page_name)


# Step 1: Initialize Wikipedia API
# Step 3: Check if the page exists
if not page.exists():
    print(f"Page '{page_title}' does not exist.")
else:
    # Step 4: Extract the content of the page
    print(f"Page - Title: {page.title}")
    
    # Get the full text of the page
    content = page.text
    
    # Step 5: Extract specific sections (e.g., 'Climate' section)
    climate_section = None
    for section in page.sections:
        if 'Climate' in section.title:
            climate_section = section
            break

    # Step 6: Print the content of the 'Climate' section (if found)
    if climate_section:
        print(f"\n--- {climate_section.title} Section ---")
        print(climate_section.text[:1000])  # Printing only the first 1000 characters for brevity
    else:
        print("Climate section not found.")

Page - Title: Denver
Climate section not found.
